<div style="text-align: center;">
    <img src=https://d1.awsstatic.com/logos/aws-logo-lockups/poweredbyaws/PB_AWS_logo_RGB_REV_SQ.8c88ac215fe4e441dc42865dd6962ed4f444a90d.png width="350" class="center">
</div>
<center> Welcome to the Scooters Graph Demo! </center>
<center><b> YouTube channel: </b> <a href="https://www.youtube.com/@awsdevelopers">AWS Developers</a></center>

## Use LangChain Generative AI, using the Neptune Open Cypher QA Chain

[LangChain](https://python.langchain.com/docs/get_started/introduction) is a framework for developing applications powered by language models. It enables applications that:

* Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)
* Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)
* The QA chain we use here, called NeptuneOpenCypherQAChain, queries Neptune graph databases using openCypher, returning human readable responses.

See more at [LangChain docs](https://python.langchain.com/docs/use_cases/graph/neptune_cypher_qa)

<div style="text-align: center;">
    <img src=https://d1.awsstatic.com/Neptune%20LangChain%20Integration.0537f77f5a3ed426c7f7aed220e6bd5e7b3ce57f.png width="650" class="center">
</div>


In [ ]:
%graph_notebook_version

In [ ]:
%graph_notebook_config

In [ ]:
%status

In [ ]:
import requests, json, traceback

<b> Requirement: </b><br>
In the next cell, change the parameter values to those from the CDK deployment output or from the AWS console:
1. Open the AWS console, go to CloudFormation and click on our CDK stack called "ScootersNeptuneStack".
2. Click on the second tab, called "Outputs".
3. Copy the value of outputneptuneendpoint and replace this to the value of NEPTUNE_SERVER_ENDPOINT
4. Copy the value of outputneptuneiamrole and replace this to the value of IAM_ROLE_ARN
5. Copy the value of outputs3bucket and replace this to the value of S3_BUCKET
6. For AWS_REGION, enter the AWS region where you're doing all this demo; e.g. eu-west-1


In [ ]:
# Input Neptune parameters, to load the generated data:
NEPTUNE_SERVER_ENDPOINT = 'change_me'
IAM_ROLE_ARN = 'change_me'
S3_BUCKET = 'change_me'
AWS_REGION = 'change_me'

In [ ]:
# Neptune endpoint. You don't need to change anything here, unless you changed the Neptune's database port.
PORT = 8182
ENDPOINT = f"https://{NEPTUNE_SERVER_ENDPOINT}:{PORT}/loader"

---

### Let's now Use LangChain Generative AI, through the Neptune Open Cypher QA Chain

In [ ]:
! pip install langchain_community langchain

In [ ]:
from langchain_community.graphs import NeptuneGraph
from langchain.chains import NeptuneOpenCypherQAChain
from langchain.llms.bedrock import Bedrock

### Create the helper function to get inference parameters based on the model.

> Source / credits of this function: [Building with Amazon Bedrock and LangChain](https://catalog.workshops.aws/building-with-amazon-bedrock/en-US/foundation/bedrock-inference-parameters)

- Each model provider has its own set of inference parameters. This function returns a set of custom parameters based on the first portion of each model's id.

- You can experiment with different inference parameters using the Text playground in the Bedrock console. You can use the View API request button in the playground to get the parameters to use in your code.

In [ ]:
def get_inference_parameters(model):
    """
    Return a default set of parameters based on the model's provider
    """
    bedrock_model_provider = model.split('.')[0] #grab the model provider from the first part of the model id
    
    if (bedrock_model_provider == 'anthropic'): #Anthropic model
        return { #anthropic
            "max_tokens_to_sample": 512,
            "temperature": 0, 
            "top_k": 250, 
            "top_p": 1, 
            "stop_sequences": ["\n\nHuman:"] 
           }
    
    elif (bedrock_model_provider == 'ai21'): #AI21
        return { #AI21
            "maxTokens": 512, 
            "temperature": 0, 
            "topP": 0.5, 
            "stopSequences": [], 
            "countPenalty": {"scale": 0 }, 
            "presencePenalty": {"scale": 0 }, 
            "frequencyPenalty": {"scale": 0 } 
           }
    
    elif (bedrock_model_provider == 'cohere'): #COHERE
        return {
            "max_tokens": 512,
            "temperature": 0,
            "p": 0.01,
            "k": 0,
            "stop_sequences": [],
            "return_likelihoods": "NONE"
        }
    
    elif (bedrock_model_provider == 'meta'): #META
        return {
            "temperature": 0,
            "top_p": 0.9,
            "max_gen_len": 512
        }

    else: #Amazon
        #For the LangChain Bedrock implementation, these parameters will be added to the 
        #textGenerationConfig item that LangChain creates for us
        return { 
            "maxTokenCount": 512, 
            "stopSequences": [], 
            "temperature": 0, 
            "topP": 0.9 
        }

#### Let's now query our graph, using natural language via LangChain
<b> IMPORTANT: </b> Don't forget [to grant your account](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) with LLM model access.

In [ ]:
# Connection:
host = NEPTUNE_SERVER_ENDPOINT
port = PORT
use_https = True

# Get model inference parameters. Feel free to change the model.
model = 'anthropic.claude-v2:1'
model_kwargs = get_inference_parameters(model)

# Model setup
graph = NeptuneGraph(host=host, port=port, use_https=use_https)
llm = Bedrock(
    region_name="us-west-2",
    model_id=model,
    model_kwargs = model_kwargs
)

# Let's use NL to ask our LLM to traverse the graph
chain = NeptuneOpenCypherQAChain.from_llm(llm=llm, graph=graph)

### Let's interrogate our graph!

In [ ]:
chain.invoke("how many scooters do I have?")

In [ ]:
# Prompt
prompt_template="""
    Human: Please, respond the question directly. The data model for our database graph follows the following guidelines:
    - All scooters are labelled as scooter.
    - All the labels for the edges is 'has'
    - Entering a dash symbol "-" as the edge, the labels for the nodes are connected in the following way:
    
    scooter - incident - legal_case
    
    question: how many scooters do I have that have had an incident?

    Assistant:"""

# Let's now ask a more-complex question:
chain.invoke(prompt_template)

In [ ]:
%%gremlin

// Confirm the LLM output. This query will count connected Scooters that had an incident.
g.V().hasLabel('scooter').repeat(__.out().simplePath()).until(hasLabel('incident')).out().count()

### Let's now see what is LangChain executing internally

In [ ]:
# Let's modify the LLM config parameters
chain = NeptuneOpenCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, top_k=10, return_direct=False)

In [ ]:
# Count scooters and show OpenCypher internal query executed by LLM
chain.invoke("""
    Human: Please, respond the question directly. The data model for our database graph follows the following guidelines:
    - All scooters are labelled as scooter.
    - All the labels for the edges is 'has'
    - Entering a dash symbol "-" as the edge, the labels for the nodes are connected in the following way:
    
    scooter - incident - legal_case
    
    question: how many scooters do I have?
    """)

In [ ]:
# Prompt
prompt_template="""
    Human: Please, respond the question directly. The data model for our database graph follows the following guidelines:
    - All scooters are labelled as scooter.
    - All the labels for the edges is 'has'
    - All parts are labelled as part_<<something>>, where <<something>> can be suspension, tyres (back_tyre or front_tyre), steering, etc.
      For example, for a tyre the label will be part_tyre or for a suspension the label will be part_suspension. 
    - Entering a dash symbol "-" as the edge, the labels for the nodes are connected in the following way:
    
    scooter - part - fault - claim
    
    question: how many suspension parts do I have that have any fault?

    Assistant:"""

# Let's now ask a more-complex question:
chain.invoke(prompt_template)

In [ ]:
%%gremlin

// Confirm the LLM output. This query will count connected Scooters that have faulty suspensions
g.V().hasLabel('part_suspension').repeat(__.out().simplePath()).until(hasLabel('fault')).count()

## Thanks!

If you want to learn about Simplifying Graph Queries With LLMs and LangChain, I recommend you to visit [this video from our YouTube channel](https://www.youtube.com/watch?v=B7GtC1IeIUA). In here, Kelvin Lawrence, graph architect and author of the fantastic [online book "Practical Gremlin"](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html), will dive deep into this topic using a popular and public Airports graph dataset. 